In [15]:
import numpy as np

import sys
from pathlib import Path

import torch

from sklearn.model_selection import train_test_split
import wandb

In [16]:
# Importerer spaleron og micro svarte hull data

module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
import trainers
import nnmodels

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [17]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [18]:
print(bhArray.shape)
print(sphArray.shape)

(15000, 50, 50, 3)
(15000, 50, 50, 3)


In [19]:
# Vi slår sammen dataene for å ha et samlet datasett som kjører gjennom modellen
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [20]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [21]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [22]:
# 75% av datasettet havner i trainData, 25% havner i testData, 
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [23]:
# gjør det om til en tensor fra numpy array. Vi gjør dette for at at dataene skal være lagret på GPU en istedet for en liste som er lagret på CPUen 
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [24]:
# to tensor datasett 
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [25]:
from torchvision import models
from torchsummary import summary

model = nnmodels.ConvModel(0).to(device)
summary(model, (3, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 48, 48]             448
            Conv2d-2           [-1, 64, 22, 22]           9,280
            Conv2d-3            [-1, 256, 9, 9]         147,712
            Linear-4                  [-1, 128]         295,040
           Dropout-5                  [-1, 128]               0
            Linear-6                    [-1, 2]             258
Total params: 452,738
Trainable params: 452,738
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.68
Params size (MB): 1.73
Estimated Total Size (MB): 2.43
----------------------------------------------------------------


In [26]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        'model': {'values': ['resnet34']},
        'loss': {'values': ['cross','customLoss-3','customLoss-5','customLoss-7']},
        'epoch': {'values': [5]},
        'batch_size': {'values': [50]},
        'gamma': {'values': [1]},
        'learning_rate': {'values': [0.01]},
        'optimizer': {'values': ['adam']},
        'dropout': {'values': [0]},
     }
}

In [27]:
#wrapper function
def trainFunction():
    with wandb.init(project="Sweep", name="Sweep") as run:
        config = wandb.config

        name_str = config['model'] + "_" + config['loss'] + "_" + config['optimizer']
        run.name = name_str
        
        trainers.sweep(train, test, config)

In [28]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project="Sweep")
wandb.agent(sweep_id, function=trainFunction)

Create sweep with ID: mf175aif
Sweep URL: https://wandb.ai/g13hvl2023/Sweep/sweeps/mf175aif


wandb: Agent Starting Run: pk4fxkny with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 5
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: cross
wandb: 	model: resnet34
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


BH accuracy,▁▃█▇██
SPH accuracy,▁█████
Test accuracy,▁▇████
Test epoch_loss,█▃▂▁▁▁
Test precision,▁█████
Test recall,▁▇████
Train accuracy,▁▇███
Train epoch_loss,█▂▁▁▁
BH accuracy,0.88684
SPH accuracy,0.94419
Test accuracy,0.9152


wandb: Agent Starting Run: 5j74r4sg with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 5
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: customLoss-3
wandb: 	model: resnet34
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


BH accuracy,▁▇██▅█
SPH accuracy,▁▇▇▇█▇
Test accuracy,▁█████
Test epoch_loss,█▂▁▁▂▁
Test precision,▁▇▇▇█▇
Test recall,▁▇██▆█
Train accuracy,▁▇▇██
Train epoch_loss,█▂▂▁▁
BH accuracy,0.93089
SPH accuracy,0.89323
Test accuracy,0.91227


wandb: Agent Starting Run: 0dgzac0d with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 5
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: customLoss-5
wandb: 	model: resnet34
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


BH accuracy,▁█▄▆▆▅
SPH accuracy,▄▁█▇▇█
Test accuracy,▁▃▇███
Test epoch_loss,▃█▁▁▁▁
Test precision,▄▁█▇▇█
Test recall,▁█▅▆▇▆
Train accuracy,▄▁▇██
Train epoch_loss,▅█▃▂▁
BH accuracy,0.82669
SPH accuracy,0.94095
Test accuracy,0.8832


wandb: Agent Starting Run: v7cc0mvn with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 5
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: customLoss-7
wandb: 	model: resnet34
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


BH accuracy,▆▁▂▆██
SPH accuracy,▁███▇▆
Test accuracy,▁▄▅▇█▇
Test epoch_loss,█▅▅▂▁▄
Test precision,▁███▇▆
Test recall,▁▃▄▆█▇
Train accuracy,▁▃▆█▇
Train epoch_loss,█▆▃▁▃
BH accuracy,0.823
SPH accuracy,0.8137
Test accuracy,0.8184


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
